In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
os.chdir('/content/drive/MyDrive/ERA_V2/Assignment9')

In [3]:
from __future__ import print_function
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [1]:
import torch
from torchvision import datasets, transforms
import numpy as np
import albumentations as A

# Define mean and standard deviation of your dataset
mean = (0.5, 0.5, 0.5)
std = (0.5, 0.5, 0.5)

# Define augmentation transforms
augmentations = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=10),
    A.CoarseDropout(max_holes=1, max_height=16, max_width=16, min_holes=1, min_height=16, min_width=16, fill_value=mean)
    ])
                     # Ensure that augmentations are always applied


# Train Phase transformations
def albumentations_transform(image):
    np_image = np.array(image)
    augmented = augmentations(image=np_image)
    return augmented['image']

train_transforms = transforms.Compose([
    albumentations_transform,
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Test Phase transformations
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])


In [3]:
train = datasets.CIFAR10('/content/drive/MyDrive/ERA_V2/Assignment9/data', train=True, download=True, transform=train_transforms)
test = datasets.CIFAR10('/content/drive/MyDrive/ERA_V2/Assignment9/data', train=False, download=True, transform=test_transforms)

#train = datasets.CIFAR10('./data', train=True, download=True, transform=train_transforms)
#test = datasets.CIFAR10('./data', train=False, download=True, transform=test_transforms)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=64, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

CUDA Available? True


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(




---



In [6]:
from model import Net


!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
              ReLU-2           [-1, 32, 32, 32]               0
       BatchNorm2d-3           [-1, 32, 32, 32]              64
           Dropout-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 64, 16, 16]             576
            Conv2d-6           [-1, 64, 16, 16]           4,096
              ReLU-7           [-1, 64, 16, 16]               0
       BatchNorm2d-8           [-1, 64, 16, 16]             128
           Dropout-9           [-1, 64, 16, 16]               0
           Conv2d-10            [-1, 128, 8, 8]          73,728
             ReLU-11            [-1, 128, 8, 8]               0
      BatchNorm2d-12            [-1, 128, 8, 8]             256
          Dropout-13            [-1, 128, 8, 8]               0
           Conv2d-14            [-

In [7]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))

In [9]:
from torch.optim.lr_scheduler import StepLR
from torch import optim

model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)


EPOCHS = 30
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(model, device, test_loader)

EPOCH: 0


Loss=1.8349342346191406 Batch_id=781 Accuracy=41.70: 100%|██████████| 782/782 [00:20<00:00, 37.26it/s]



Test set: Average loss: 1.4795, Accuracy: 5141/10000 (51.41%)

EPOCH: 1


Loss=1.670573115348816 Batch_id=781 Accuracy=53.22: 100%|██████████| 782/782 [00:20<00:00, 37.45it/s]



Test set: Average loss: 1.2605, Accuracy: 5732/10000 (57.32%)

EPOCH: 2


...


Test set: Average loss: 0.8029, Accuracy: 7290/10000 (72.90%)

EPOCH: 28


Loss=1.1075124740600586 Batch_id=781 Accuracy=70.72: 100%|██████████| 782/782 [00:25<00:00, 30.50it/s]



Test set: Average loss: 0.8085, Accuracy: 7255/10000 (72.55%)

EPOCH: 29


Loss=1.7344717979431152 Batch_id=781 Accuracy=70.98: 100%|██████████| 782/782 [00:23<00:00, 33.22it/s]



Test set: Average loss: 0.8100, Accuracy: 7262/10000 (72.62%)

